In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# PARAMETERS
train_dir = r'C:\Users\kashi\Desktop\LY Project\Multi Cancer\Robust\Robust_Data'  # Dataset path
img_height, img_width = 299, 299  # InceptionV3 input size
batch_size = 32
num_classes = 16  # Update if number of classes differs
epochs = 20

# LOAD DATA - RESCALE AND SPLIT
print("Loading training data without augmentation (only rescaling)...")

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% validation split
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# LOAD InceptionV3 BASE MODEL AND FREEZE LAYERS
print("Loading InceptionV3 pre-trained base model...")
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in base_model.layers:
    layer.trainable = False
print(f"Frozen {len(base_model.layers)} layers of InceptionV3 base.")

# ADD CUSTOM CLASSIFIER ON TOP
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# COMPILE MODEL
print("Compiling model...")
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# TRAIN MODEL WITH CHECKPOINT
checkpoint_filepath = 'best_inceptionv3_cancer_model.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, monitor='val_accuracy', save_best_only=True, verbose=1
)

print(f"Training for {epochs} epochs...")
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=[checkpoint]
)

print("Training completed.")
print(f"Best model saved at {checkpoint_filepath}")

# LOAD BEST MODEL FROM CHECKPOINT
print("Loading best model for evaluation...")
model.load_weights(checkpoint_filepath)

# PREDICTIONS AND CONFUSION MATRIX
print("Predicting classes on validation data...")
val_steps = validation_generator.samples // validation_generator.batch_size + 1

y_pred_probs = model.predict(validation_generator, steps=val_steps)
y_pred = np.argmax(y_pred_probs, axis=1)

y_true = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))


Dataset URL: https://www.kaggle.com/datasets/revanatu/histopathological-cancer-dataset-robust


KeyboardInterrupt: 